# Writing data to the Vector Database

Prerequites:

1. This notebook assumes you have a Weaviate instance running. https://weaviate.io/developers/weaviate/installation
2. Have a commercial Azure openAI endpoint provisioned and model deployed. 

This demo takes a locally stored txt file, splits it, and writes to weaviate given an embedding.

Below are a list of necessary packages:
- weviate-client is for connecting to weaviate from python. 
- tiktoken is a dependancy for OpenAIEmbeddings.
- openai is for connecting to your AOAI instance.
- langchain is a language model integration framework. 

In [ ]:
%pip install weaviate-client
%pip install tiktoken
%pip install openai[datalib]
%pip install langchain
%pip install arxiv
%pip install pymupdf

## Load Data from Arxiv and Clean

In [ ]:
from langchain.document_loaders import ArxivLoader
docs = ArxivLoader(query="0704.0001", load_max_docs=2).load()
len(docs)

In [ ]:
docs[0].metadata  # meta-information of the Document

In [ ]:
#Need to add cell here that takes Arxiv data and transforms the metadata, most likely using: https://python.langchain.com/docs/integrations/document_transformers/doctran_extract_properties

## Setting our embedding parameters. 

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(
    deployment="textembedding",
    model="text-embedding-ada-002",
    openai_api_base="https://aoaivbd.openai.azure.com/",
    openai_api_type="azure",
    openai_api_key="7ad1367445ed4388932ac7c5edd32dd0"
)

## Write to authenticated Weaviate

Note: To use internal IP of the VM hosting docker instance compute instance has to be in same vnet. 
Example: https://10.0.0.4:8080

In [ ]:
import weaviate

WEAVIATE_URL = "http://10.244.3.3:8080"
WEAVIATE_API_KEY = "TJVA95OrM7E20RMHrHDcEfxjoYZgeFONFh7HgQ"

client = weaviate.Client(url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))
vectorstore = Weaviate.from_documents(docs, embeddings, client=client, by_text=False)

In [ ]:
# Add Query to Verify
#query = "What did the president say about Ketanji Brown Jackson"
#doc = vectorstore.similarity_search(query)